In [2]:
#****************************** Step1 ********************************
# Imporing Standard and third-party imports for RAG pipeline
import pandas as pd
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import pipeline
import torch

c:\Users\ltlid\OneDrive\Desktop\Educational\10 Academy\Week 6\Intelligent_complain_analysis\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
#************************ Step 2 *********************************************************
# *********************** Load FAISS Vectore Store and Embedding Model *******************

# Load the same embedding model as used in Task 2
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Load the vector store built in Task 2
vector_db = FAISS.load_local(
    "../notebooks/vector_store",  
    embedding_model,
    allow_dangerous_deserialization=True
)


RuntimeError: Error in __cdecl faiss::FileIOReader::FileIOReader(const char *) at D:\a\faiss-wheels\faiss-wheels\faiss\faiss\impl\io.cpp:68: Error: 'f' failed: could not open ..\notebooks\vector_store\index.faiss for reading: No such file or directory

In [ ]:
#********************************* Step 3 *******************************************
# ********************************Define Retrival Function ***************************

def retrieve_top_chunks(query: str, k: int = 5):
    """
    Retrieves the top-k most relevant complaint chunks from FAISS index.
    
    Args:
        query (str): The user's question.
        k (int): Number of chunks to retrieve.
        
    Returns:
        List of Document objects.
    """
    return vector_db.similarity_search(query, k=k)

In [ ]:
#***************************************  Step 4 *********************************
#***************************************  Define RAG Prompt Template *************

rag_prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are a financial analyst assistant at CrediTrust.
Your task is to answer customer complaint-related questions using only the provided context.

Context:
{context}

Question:
{question}

Answer (based only on the context above):
"""
)


In [ ]:
#************************* Step 5 ****************************************
# ************************ Load a Lightweight LLM ************************

# We'll use HuggingFace's DistilBERT for generation (very lightweight and CPU-friendly)

llm_pipeline = pipeline(
    "text-generation",
    model="distilgpt2",  # This can be switch to mistral, llama, etc. concidering available resource
    max_new_tokens=200,
    temperature=0.7,
    do_sample=True
)

llm = HuggingFacePipeline(pipeline=llm_pipeline)

In [ ]:
#********************** Step 6 **************************
#********************* Combine Retriever + LLM into RetrievalQA Chain ********************

rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vector_db.as_retriever(search_kwargs={"k": 5}),
    chain_type="stuff",
    chain_type_kwargs={"prompt": rag_prompt_template}
)

In [ ]:
# ************************* Step 7 *********************************************
# ************************* Define Example Queries for Evaluation **************

example_questions = [
    "Why are users unhappy with Buy Now, Pay Later services?",
    "What issues do customers report about savings accounts?",
    "Are there any complaints about money transfers being delayed?",
    "Why do people dislike credit cards?",
    "What is the most common complaint about personal loans?"
]

In [ ]:
# ************************** Step 8 *********************************************
# ************************** Run Evaluation and Store Results *******************

evaluation_results = []

for question in example_questions:
    result = rag_chain({"query": question})
    retrieved_docs = retrieve_top_chunks(question)
    
    evaluation_results.append({
        "Question": question,
        "Generated Answer": result["result"],
        "Retrieved Source 1": retrieved_docs[0].page_content[:250],  # Short preview
        "Retrieved Source 2": retrieved_docs[1].page_content[:250] if len(retrieved_docs) > 1 else "",
        "Quality Score (1-5)": "",  # Leave blank to score manually
        "Comments": ""
    })

In [ ]:
# ********************* Step 9 ********************************************************
# ********************* Display Evaluation Table for Manual Review ********************

eval_df = pd.DataFrame(evaluation_results)
eval_df[["Question", "Generated Answer", "Retrieved Source 1", "Retrieved Source 2", "Quality Score (1-5)", "Comments"]]
